## Logistic loss
Logistic loss measures how well a model’s predicted probabilities match the actual binary outcomes (0 or 1). It penalizes confident but incorrect predictions very heavily.

In [ ]:
import numpy as np

def logistic_loss(Y, y_pred_proba):
    loss = -(Y * np.log(y_pred_proba) + (1 - Y) * np.log(1 - y_pred_proba))
    return np.mean(loss)

## Calibration
Model calibration refers to how well a model’s predicted probabilities reflect the true likelihood of outcomes.

In a well-calibrated model, predictions match observed frequencies.

Calibration model is learning how to model: raw probability → true empirical probability

1. Train base classifier on training data
2. Predict probabilities on calibration data
3. Fit isotonic or logistic calibrator (X - predicted probs ; Y - true class)
4. Apply calibrator to test predictions
5. Evaluate using Brier score or calibration curve
6. Optionally compute classification metrics

In [ ]:
# Pipeline for calibrating a model
model = (...)
model.fit(X_train, Y_train)

y_pred_proba = model.predict_proba(X_calib)

calib_model = (...)
calib_model.fit(y_pred_proba.reshape(-1, 1), Y_calib)

pred_raw = model.predict_proba(X_test)
pred_calib = calib_model.predict_proba(pred_raw.reshape(-1,1))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

prob_true_raw, prob_pred_raw = calibration_curve(Y_test, raw_probs, n_bins=10, strategy='uniform')
prob_true_cal, prob_pred_cal = calibration_curve(Y_test, calibrated_probs, n_bins=10, strategy='uniform')

# Plot
plt.figure(figsize=(8, 6))
plt.plot(prob_pred_raw, prob_true_raw, marker='o', label='Raw model')
plt.plot(prob_pred_cal, prob_true_cal, marker='s', label='Calibrated model')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly calibrated')
plt.xlabel('Predicted probability')
plt.ylabel('Observed fraction of positives')
plt.title('Calibration Plot / Reliability Diagram')
plt.legend()
plt.grid(True)
plt.show()

## Confidence Intervals
Calculating confidence intervals using:
- CLT
- Markov's inequality
- Chebychev's ineugality
- Hoeffding's inequality
- Bennet's inequality

In [ ]:
# CLT
import numpy as np
from scipy import stats

def conf_interval(data, conf_level = 0.95):
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)

    # CI using t-distribution
    alpha = 1 - conf_level
    t_crit = stats.t.ppf(1 - alpha/2, df=n-1)

    margin_error = t_crit * std / np.sqrt(n)
    ci = (mean - margin_error, mean + margin_error)
    print("Sample mean:", mean)
    print("95% CI:", ci)


Sample mean: 6.444444444444445
95% CI: (np.float64(5.163331035997686), np.float64(7.725557852891203))


In [16]:
# Markov's inequality
import numpy as np

def markov_conf_interval(data, conf_level=0.95):
    n = len(data)

    sample_mean = np.mean(data)
    alpha = 1 - conf_level
    upper_bound = sample_mean / alpha

    return upper_bound


In [17]:
def chebychev_conf_interval(data, conf_level=0.95):
    n = len(data)
    alfa = 1 - conf_level
    epsilon = np.sqrt(np.var(data)/(n*alfa))
    mean = np.mean(data)

    return (mean - epsilon, mean + epsilon)

In [18]:
import numpy as np

def hoeffding_conf_interval(data, a, b, conf_level=0.95):
    alpha = 1 - conf_level
    n = len(data)
    epsilon = (b - a) / np.sqrt(2 * n) * np.sqrt(np.log(2 / alpha))
    mean = np.mean(data)
    return (mean - epsilon, mean + epsilon)


In [19]:
import numpy as np

def bennet_conf_interval(data, conf_level=0.95):
    variance = np.var(data)
    alfa = 1 - conf_level
    n = len(data)

    epsilon = np.sqrt( (2 * variance * np.log(2/alfa)) / n ) + (5 * np.log(2/alfa)) / (3 * n)
    mean = np.mean(data)

    interval = (mean - epsilon, mean + epsilon)
    return interval

In [ ]:
import numpy as np

def makeFreq(data_sequence):
    """
    Takes a data_sequence in the form of iterable and returns a
    numpy array of the form [keys,counts] where the keys
    are the unique values in data_sequence and counts are how
    many time they appeared
    """
    import numpy as np
    data = np.array(data_sequence)
    if (len(data.shape) == 2):
        (keys,counts) = np.unique(data.T,axis=0,return_counts=True)
        return np.concatenate([keys,counts.reshape(-1,1)],axis=1)
    else:
        (keys,counts) = np.unique(data,return_counts=True)
        return np.stack([keys,counts],axis=-1)

def makeEDF(data_sequence):
    import numpy as np
    numRelFreqPairs = makeFreq(data_sequence)
    (keys,counts) = (numRelFreqPairs[:,0],numRelFreqPairs[:,1])
    frequencies = counts/np.sum(counts)
    emf = np.stack([keys,frequencies],axis=-1)
    cumFreqs = np.cumsum(frequencies)
    edf = np.stack([keys,cumFreqs],axis=-1)

    return edf

def DKW_conf_interval(data, conf_level=0.95):
    edf = makeEDF(data.reshape(-1))
    alpha = 1 - conf_level
    n = len(data)
    epsilon = np.sqrt(np.log(2 / alpha) / (2 * n))
    lower = np.maximum(0, edf - epsilon)
    upper = np.minimum(1, edf + epsilon)
    return (lower, upper)

## Markov chains

In [ ]:
# calculating Stationary Distribution
import numpy as np

def stationary_dist(P):
    P = np.array(P)
    eigenvals, eigenvecs = np.linalg.eig(P.T)

    # find the index of the eigenvalue that is 1
    index = np.argmin(np.abs(eigenvals - 1))

    stat = eigenvecs[:, index].flatten() # getting eigenvec with eigenval 1
    stat = stat / np.sum(stat) #normalizing vector
    return stat

In [ ]:
# calculating if the MC is irreducible
import numpy as np
import networkx as nx

def is_irreducible(P):
    G = nx.DiGraph()
    n = P.shape[0]
    
    for i in range(n):
        for j in range(n):
            if P[i, j] > 0:
                G.add_edge(i, j)
    
    return nx.is_strongly_connected(G)

def is_irreducible_nx(matrix):
    # Convert numpy array to a NetworkX Directed Graph
    G = nx.from_numpy_array(matrix, create_using=nx.DiGraph)
    
    # Check if the graph is strongly connected
    return nx.is_strongly_connected(G)

In [ ]:
# calculating if the MC is aperiodic
from math import gcd
from functools import reduce

def is_aperiodic(P, max_power=100):
    n = P.shape[0]
    powers = np.eye(n)
    return_times = []

    for k in range(1, max_power + 1):
        powers = powers @ P
        if powers[0, 0] > 0:  # return to state 0
            return_times.append(k)

    if len(return_times) == 0:
        return False

    period = reduce(gcd, return_times)
    return period == 1


In [ ]:
# calculating if the MC is reversible
def is_reversible(P, tol=1e-8):

    if not (is_irreducible(P) and is_aperiodic(P)): # if there is no stationary distribution
        return False
    
    P = np.array(P)
    n = P.shape[0]

    pi = stationary_dist(P)

    for i in range(n):
        for j in range(n):
            if not np.isclose(pi[i] * P[i, j], pi[j] * P[j, i], atol=tol):
                return False

    return True

In [ ]:
# calculating n-step transition matrix
import numpy as np

def n_step_transition(P, n):
    '''
    Calculating n-step transition matrix.
    P^n --> transitions after n steps.
    With increasing n, each row will converge to stationary dist.
    '''
    return np.linalg.matrix_power(P, n)

In [ ]:
def comp_transition_matrix(transitions, n_states):
    """
    Estimate the transition matrix P from observed transitions.

    Args:
        transitions: array of shape (n_samples, 2)
        n_states: number of states

    Returns:
        P_hat: estimated transition matrix
    """
    P_hat = np.zeros((n_states, n_states))
    
    n_transitions = transitions.shape[0]
    for t in transitions:
        
        P_hat[t[0],t[1]] += 1

    row_sums = P_hat.sum(axis=1)
    P_hat = P_hat / row_sums[:, np.newaxis]

    return P_hat

In [ ]:
def is_transition_matrix(P):
    """
    Check if P is a transition matrix.
    """

    if P.ndim != 2 or P.shape[0] != P.shape[1]:
        print("Wrong shape")
        return False

    if not np.isfinite(P).all():
        print("Is not finite")
        return False

    if np.all(P < 0):
        print("Has negative elements")
        return False

    if not np.allclose(P.sum(axis=1), 1.0, atol=1e-8):
        print("Rows do not sum to 1")
        return False
    
    return True

In [ ]:
def simulate_chain(P, start_state, n_steps):
    """
    Simulate a Markov chain trajectory with a fixed random seed.

    Returns: array of visited states of length n_steps + 1
    """
    # seed = 1234  # don't change that
    rng = np.random.default_rng()

    path = np.zeros(n_steps + 1, dtype=int)
    path[0] = start_state

    for t in range(n_steps):
        path[t + 1] = rng.choice(
            P.shape[0],
            p=P[path[t]]
        )

    return path

## Linear Congruential Generator LCG

This is a  simple way to generate chains of pseudorandom numbers and uniform distributions

A good way to create an LCG is to follow the hull dobell theorem
 ### Hull–Dobell Theorem.

The congruential generator (a,b,M) has period M iff

1. gcd(b,M)=1,

2. p divides a-1 for every prime p that divides M

3. 4 divides a- 1 if 4 divides M.

some good numbers for LCG:

m = 2^31 - 1 = 2147483647

a=16807

c = 0

seed = {1, …, m-1}

Xn+1 = a*Xn mod (2^31 - 1)
IF WE WANT UNIFORM NUMBERS Un = Xn/m

In [ ]:
def linConGen(n, m, a, b, x0):
    '''A linear congruential sequence generator.
    
    Param m is the integer modulus to use in the generator.
    Param a is the integer multiplier.
    Param b is the integer increment.
    Param x0 is the integer seed.
    Param n is the integer number of desired pseudo-random numbers.
    
    Returns a list of n pseudo-random integer modulo m numbers.'''
    
    x = x0 # the seed
    retValue = [x % m]  # start the list with x=x0
    for i in range(2, n+1, 1):
        x = (a * x + b) % m # the generator, using modular arithmetic
        retValue.append(x) # append the new x to the list
    return retValue

def uniformFromLCG(n,m,a,b,x0):
  return linConGen(n,m,a,b,x0) / m

## Accept-Reject Sampler

In [ ]:
def problem1_inversion(f, M, n_samples=1):

    if M is None: # finding max of a function
        from scipy.optimize import minimize_scalar
        res = minimize_scalar(lambda x: -f(x), bounds=(0, 1), method='bounded')
        M = f(res.x)

    result = []
    while (len(result) < n_samples):
        x1 = np.random.uniform(0,1)
        f_x = f(x1)
        x2 = np.random.uniform(0,1)
        if (x2 <= f_x/M):
            result.append(x1)
        else:
            continue
    return np.array(result)

# Inverse Transform Sampling ITS

For an exponential distribution with parameter $\lambda = 1$, the Probability Density Function (PDF) is given by:

$f(x) = e^{-x}$ for $x \ge 0$

The Cumulative Distribution Function (CDF) is found by integrating the PDF:

$F(x) = \int_{0}^{x} e^{-t} dt = [-e^{-t}]_{0}^{x} = -e^{-x} - (-e^{-0}) = 1 - e^{-x}$

To find the Inverse CDF, we set $F(x) = u$, where $u$ is a random variable uniformly distributed between 0 and 1, and solve for $x$:

$u = 1 - e^{-x}$

$e^{-x} = 1 - u$

$-x = \ln(1 - u)$

$x = -\ln(1 - u)$

This formula allows us to generate samples from an exponential distribution using uniform random numbers.

In [ ]:
#exponential distribution with lambda = 1
pdf_x = lambda x: np.exp(-x)
cdf_x = lambda x: 1 - np.exp(-x)
cdf_inverse_x = lambda u: -np.log(1 - u)

Xs = np.linspace(0, 10, num = 1000)
Ys = pdf_x(Xs)

plt.plot(Xs,Ys, label = 'true distribution')



#Inverse sampling of the distribution
samples = []
for i in range(10000):
  u = np.random.uniform(0,1)
  sample_x = cdf_inverse_x(u)
  samples.append(sample_x)


plt.hist(samples, bins = 100, density= True, label = 'samples')
plt.legend()

# Box-Muller Sampling (Box-Muller Transform)


To generate two independent samples, $Z_0$ and $Z_1$, from a Standard Normal Distribution $N(0, 1)$, follow these steps:

Step 1: Generate Uniform Inputs

Generate two independent random numbers, $U_1$ and $U_2$, from a uniform distribution:$$U_1, U_2 \sim \text{Uniform}(0, 1)$$

Step 2: Apply the Box-Muller Equations

Use the following transformation equations to convert the uniform samples into normal samples:

$$Z_0 = \sqrt{-2 \ln U_1} \cos(2\pi U_2)$$

$$Z_1 = \sqrt{-2 \ln U_1} \sin(2\pi U_2)$$

Step 3: Adjust for Target Mean ($\mu$) and Variance ($\sigma^2$)

If you need samples from a distribution $N(\mu, \sigma^2)$ rather than the standard $N(0, 1)$, apply the linear transformation:$$X = \mu + Z\sigma$$

Summary of the mapping:

The term $\sqrt{-2 \ln U_1}$ generates the magnitude (radius).The term $2\pi U_2$ generates the angle (0 to 360 degrees).The $\cos$ and $\sin$ functions project that magnitude and angle onto the $X$ and $Y$ axes to produce the final coordinates (samples).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def box_muller_sample(n_samples, mu=0, sigma=1):
    """
    Generates normal samples using the Box-Muller transform.
    Returns two arrays of independent samples.
    """
    # Step 1: Generate uniform random numbers U1, U2 ~ Uniform(0, 1)
    u1 = np.random.rand(n_samples)
    u2 = np.random.rand(n_samples)

    # Step 2: Apply the transformation equations
    # Magnitude (Radius)
    mag = np.sqrt(-2 * np.log(u1))

    # Angle (0 to 2*pi)
    angle = 2 * np.pi * u2

    # Calculate Z0 and Z1
    z0 = mag * np.cos(angle)
    z1 = mag * np.sin(angle)

    # Step 3: Scale to target mean and standard deviation
    x0 = mu + z0 * sigma
    x1 = mu + z1 * sigma

    return x0, x1

# --- Execution and Visualization ---
n = 10000
samples_x, samples_y = box_muller_sample(n)

# Plotting the results
plt.figure(figsize=(12, 5))

# Plot Histogram to verify Normal Distribution
plt.subplot(1, 2, 1)
plt.hist(samples_x, bins=50, density=True, color='skyblue', edgecolor='black', alpha=0.7)
plt.title(f"Histogram of Z0 samples (n={n})")
plt.xlabel("Value")
plt.ylabel("Frequency")

# Plot Scatter to show independence/joint distribution
plt.subplot(1, 2, 2)
plt.scatter(samples_x, samples_y, s=1, alpha=0.2, color='purple')
plt.title("Joint Distribution (Z0 vs Z1)")
plt.xlabel("Z0")
plt.ylabel("Z1")

plt.tight_layout()
plt.show()

# EDA

In [ ]:
# plotting histograms of each column in a df

n_rows, n_cols = 6, 5

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 16))  # Adjust figure size
axes = axes.flatten()  # Flatten 2D array of axes to 1D for easy iteration

for i, col in enumerate(df.columns):
    axes[i].hist(df[col], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(col)

# Hide any unused subplots if X has fewer than 16 columns
for j in range(i+1, n_rows*n_cols):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# correlation heatmap
import seaborn as sns

corr = df.corr()
sns.heatmap(corr, annot=True)

# Poisson Regression - evaluation

This is poisson deviance used for evaluation of Poisson Regression.

In [ ]:
import numpy as np

def poisson_deviance(y_true, y_pred, eps=1e-10):
    """
    Oblicza dewiancję Poissona

    y_true : array-like (prawdziwe liczby wizyt)
    y_pred : array-like (przewidywane lambda > 0)
    eps    : zabezpieczenie numeryczne dla log(0)
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # zabezpieczenie: lambda musi być dodatnia
    y_pred = np.maximum(y_pred, eps)

    # specjalne traktowanie y = 0 (bo 0 * log(0) = 0)
    term = np.where(
        y_true == 0,
        -y_pred,
        y_true * np.log(y_true / y_pred) - (y_true - y_pred)
    )

    return 2 * np.sum(term)
